# Submitted By,
#### Sangeeth S V - 181CO246
#### Rohith V S - 181CO243
#### Jayakrishna Sukumaran - 181CO223
#### J Sudarsanan - 181CO222

## This is a simulator for a 2-Pass Assembler for the uPower ISA.

In [1]:
import re
import sys

## Pass-1
##### In pass-1, we try to identify and note down all the labels and their position in the assembly level code. Their position is noted based on the line number in the assembly code and stored in 'label.txt'. This is the primary purpose of pass-1. We also split each of the instructions and identify the instruction type, and store both instruction type and the instruction opcode in a new file, ref.txt. This is so that we can easily identify the instructions in pass-2.

###### Labels include loop beginnings, function names and also the variable names in the .data portion

In [2]:
def pass1(i):
    file=open('Q' + str(i) + '.s','r')
    file1=open('label.txt','w')
    file2=open('ref.txt','w')
    flag=0
    c=0
    str1=""
    str2=""
    str3=""
    for each in file:
        c=c+1
        if each[0]=='#':
            continue
        x=re.split(', |\n|\s|\n|"',each)
        while("" in x):
            x.remove("")
        y=len(x);
        if y==0:
            continue
        if x[0]=='.data':
            flag=1
        if x[0]=='.text':
            flag=2
            #file2.write(str(c)+'\n')
            continue

        if flag==1:
            temp=len(x[0])
            if x[0][temp-1]==':':
                str1=str1+str(c)+'#'
                str1=str1+x[0][0:temp-1]+'#'
                for temp in x[1:y]:
                    if temp=='.asciiz' or temp=='.word':
                        str1=str1+temp+'#'
                    else:
                        str1=str1+temp+' '
                file1.write(str1+'\n')
                str1=""

        if flag==2:
            y=len(x[0])
            if x[0][y-1]==':':
                str1=str1+str(c)+'#'
                str1=str1+x[0][0:y-1]+'#'
                file1.write(str1+'\n')
                str1=""
                continue

            str2=str2+str(c)+'#'
            if x[0]=='add':
                str2=str2+'XO'+'#'+'011111'+'#'+'01000010100'
            elif x[0]=='addi':
                str2=str2+'D'+'#'+'001110'
            elif x[0]=='addis':
                str2=str2+'D'+'#'+'001111'
            elif x[0]=='and':
                str2=str2+'X'+'#'+'011111'+'#'+'00000111000'
            elif x[0]=='andi':
                str2=str2+'D'+'#'+'011100'
            elif x[0]=='extsw':
                str2=str2+'X'+'#'+'011111'+'#'+'11000111000'
            elif x[0]=='nand':
                str2=str2+'X'+'#'+'011111'+'#'+'01110111000'
            elif x[0]=='or':
                str2=str2+'X'+'#'+'011111'+'#'+'01101111000'
            elif x[0]=='ori':
                str2=str2+'D'+'#'+'011000'
            elif x[0]=='subf':
                str2=str2+'XO'+'#'+'011111'+'#'+'00001010000'
            elif x[0]=='xor':
                str2=str2+'X'+'#'+'011111'+'#'+'00100111100'
            elif x[0]=='xori':
                str2=str2+'D'+'#'+'011010'
            elif x[0]=='ld':
                str2=str2+'DS'+'#'+'111010'+'#'+'00'
            elif x[0]=='lwz':
                str2=str2+'D'+'#'+'100000'
            elif x[0]=='std':
                str2=str2+'DS'+'#'+'111110'+'#'+'00'
            elif x[0]=='stw':
                str2=str2+'D'+'#'+'100100'
            elif x[0]=='stwu':
                str2=str2+'D'+'#'+'100101'
            elif x[0]=='lhz':
                str2=str2+'D'+'#'+'101000'
            elif x[0]=='lha':
                str2=str2+'D'+'#'+'101010'
            elif x[0]=='sth':
                str2=str2+'D'+'#'+'101100'
            elif x[0]=='lbz':
                str2=str2+'D'+'#'+'100010'
            elif x[0]=='stb':
                str2=str2+'D'+'#'+'100100'
            elif x[0]=='rlwinm':
                str2=str2+'M'+'#'+'010101'+'#'+'0'
            elif x[0]=='sld':
                str2=str2+'X'+'#'+'011111'+'#'+'00000110110'
            elif x[0]=='srd':
                str2=str2+'X'+'#'+'011111'+'#'+'10000110110'
            elif x[0]=='srad':
                str2=str2+'X'+'#'+'011111'+'#'+'11000110100'
            elif x[0]=='sradi':
                str2=str2+'XS'+'#'+'011111'+'#'+'11001110100'
            elif x[0]=='b':
                str2=str2+'I'+'#'+'010010'+'#'+'00'
            elif x[0]=='ba':
                str2=str2+'I'+'#'+'010010'+'#'+'10'
            elif x[0]=='bl':
                str2=str2+'I'+'#'+'010010'+'#'+'01'
            elif x[0]=='bc':
                str2=str2+'B'+'#'+'010011'+'#'+'00'
            elif x[0]=='bca':
                str2=str2+'B'+'#'+'010011'+'#'+'10'
            elif x[0]=='bclr':
                str2=str2+'XL'+'#'+'010011'+'#'+'1'
            elif x[0]=='cmp':
                str2=str2+'X'+'#'+'011111'+'#'+'00000000000'
            elif x[0]=='cmpi':
                str2=str2+'D'+'#'+'001101'
            elif x[0]=='sc':
                str2=str2+'SC'+'#'+'010001'
            else:
                str2=str2+'P'+'#'+x[0]
            file2.write(str2+'\n')
            str2=""
    file.close()
    file1.close()
    file2.close()

## Pass-2
##### Pass-2 gets the offset for all the labels, and the register values accessed in each instruction. Using this, and the opcode generated from pass-1, pass-2 will store all the 32b instructions one after the other in a new file named, 'opcode.o'. The '.o' extension is for object files that are the files that get executed. 

###### Also it generates an 'opcodeindex.txt' file which stores the line numbers of all the instructions along with the 32b opcode. This is so that, in case of looping statements the control can be efficiently shifted back to the start of the loop after every iteration.


##### This function 'binary', takes two inputs, n and m, and returns a binary value in string data type with length, m. This will be useful to fill in the leading zeroes wherever necessary in the 32bit binary code.

In [3]:
def pass2(i):
    def binary(n,m):
        x=bin(n)
        x=x[2:]
        while len(x)<m:
            x='0'+str(x)
        return x

    def form(i,flag):
        f3=open('label.txt','r')
        temp=re.split('#|\n|(|)|,',i) #temp contains the line numbers of all the instructions and type of instruction along with instruction mnemonic
        while "" in temp:
            temp.remove("")
        while  None in temp:
            temp.remove(None)
        x=int(temp[0])
        x=x-1
        while x!=0:
            x=x-1
            line=f.readline()
        line=f.readline()
        arr=re.split(', |\n|\s|\(|\)|,|R',line) #arr contains all the things in one line of assembly code
        while "" in arr:
            arr.remove("")
        while  None in arr:
            arr.remove(None)
        f.seek(0,0)
        f3.seek(0,0)

        p=temp[1]
        str1=""
        if p!='XL':
            if arr[1]=='4':
                flag=0
            elif arr[1]=='3':
                flag=1
        if p=='XO':
            str1=str1+temp[2]+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str(binary(int(arr[3]),5))+temp[3]
        elif p=='X':
            str1=str1+temp[2]+str(binary(int(arr[2]),5))+str(binary(int(arr[1]),5))+str(binary(int(arr[3]),5))+temp[3]
        elif p=='D':
            str1=str1+temp[2]+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str(binary(int(arr[3]),14))+'00'
        elif p=='DS':
            str1=str1+temp[2]+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str(binary(int(arr[3]),14))+temp[3]
        elif p=='XS':
            str1=str1+temp[2]+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str(binary(int(arr[3]),5))+temp[3]
        elif p=='M':
            str1=str1+temp[2]+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str(binary(int(arr[3]),5))+str(binary(int(arr[4]),5))+str(binary(int(arr[5]),5))+temp[3]
        elif p=='XL':
            str1=str1+temp[2]+str(binary(0,25))+temp[3]
        elif p=='B':
            str2=""
            for each in f3:
                temp2=re.split('#',each)
                if temp2[1]==arr[3]:
                    str2=str(binary(int(temp2[0]),14))
            str1=str1+temp[2]+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str2+temp[3]
        elif p=='I':
            str2=""
            for each in f3:
                temp2=re.split('#',each)
                if temp2[1]==arr[1]:
                    str2=str(binary(int(temp2[0]),24))
            str1=str1+temp[2]+str2+temp[3]
        elif p=='SC':
            if flag==0:
                str1=str1+temp[2]+str(binary(int(arr[1]),25))+'0'
            else:
                str1=str1+temp[2]+str(binary(int(arr[1]),25))+'1'
        elif p=='P':
            if temp[2]=='li':
                str1=str1+'001110'+str(binary(int(arr[1]),5))+'00000'+str(binary(int(arr[2]),14))+'00'
            elif temp[2]=='subi':
                str1=str1+'001110'+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str(binary(int(arr[3]),14))+'10'
            elif temp[2]=='mr':
                str1=str1+'011111'+str(binary(int(arr[2]),5))+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+'01101111000'
            elif temp[2]=='la':
                str2=""
                for each in f3:
                    temp2=re.split('#',each)

                    if temp2[1]==arr[3]:
                        str2=str2+str(binary(int(temp2[0]),14))
                        break
                str1=str1+'001110'+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+str2+'01'
            elif temp[2]=='beq':
                str2=""
                for each in f3:
                    temp2=re.split('#',each)
                    if temp2[1]==arr[3]:
                        str2=str(binary(int(temp2[0]),16))
                        break
                f2.write('011111'+'00111'+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+'00000000000'+'\n')
                str1=str1+'010011'+'01100'+'01010'+str2
            elif temp[2]=='bne':
                str2=""
                for each in f3:
                    temp2=re.split('#',each)
                    if temp2[1]==arr[3]:
                        str2=str(binary(int(temp2[0]),16))
                        break
                f2.write('011111'+'00111'+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+'00000000000'+'\n')
                f4.write(temp[0]+'#'+'011111'+'00111'+str(binary(int(arr[1]),5))+str(binary(int(arr[2]),5))+'00000000000'+'\n')
                str1=str1+'010011'+'00100'+'00010'+str2
        #else:
            #print 'ERROR'
        f2.write(str1+'\n')
        f4.write(temp[0]+'#'+str1+'\n')
        f3.close()
    #end function
    f=open('Q' + str(i) + '.s','r')
    f1=open('ref.txt','r+')
    f2=open('opcode.o','w')
    f4=open('opcodeindex.txt','w')
    flag=0
    for j in f1:
        flag=form(j,flag)
    f.close()
    f1.close()
    f2.close()
    f4.close()


# Execute
##### This function takes the generated opcode.o file, obtained from the pass-1 and pass-2 and then disassembles it to execute the assembly level code.

##### It also simulates the memory status for both the data and instruction memory, based on what qtspim does. In step by step execution, we will be able to see the changes occuring in the registers live just like in qtspim

In [4]:

def execute(ch):
    R=[0 for i in range(32)]
    CIA=hex(0x400000)
    NIA=hex(0)
    SR=hex(0)
    CR='0000'
    LR=hex(0)
    f=open('memory.txt','w')
    f.write('CIA\t:\t'+str(CIA)+'\n')
    f.write('NIA\t:\t'+str(NIA)+'\n')
    f.write('SR\t:\t'+str(SR)+'\n')
    f.write('LR\t:\t'+str(LR)+'\n')
    f.write('CR\t:\t'+str(CR)+'\n')
    f.write('R0\t:\t'+str(R[0])+'\n')
    f.write('R1\t:\t'+str(R[1])+'\n')
    f.write('R2\t:\t'+str(R[2])+'\n')
    f.write('R3\t:\t'+str(R[3])+'\n')
    f.write('R4\t:\t'+str(R[4])+'\n')
    f.write('R5\t:\t'+str(R[5])+'\n')
    f.write('R6\t:\t'+str(R[6])+'\n')
    f.write('R7\t:\t'+str(R[7])+'\n')
    f.write('R8\t:\t'+str(R[8])+'\n')
    f.write('R9\t:\t'+str(R[9])+'\n')
    f.write('R10\t:\t'+str(R[10])+'\n')
    f.write('R11\t:\t'+str(R[11])+'\n')
    f.write('R12\t:\t'+str(R[12])+'\n')
    f.write('R13\t:\t'+str(R[13])+'\n')
    f.write('R14\t:\t'+str(R[14])+'\n')
    f.write('R15\t:\t'+str(R[15])+'\n')
    f.write('R16\t:\t'+str(R[16])+'\n')
    f.write('R17\t:\t'+str(R[17])+'\n')
    f.write('R18\t:\t'+str(R[18])+'\n')
    f.write('R19\t:\t'+str(R[19])+'\n')
    f.write('R20\t:\t'+str(R[20])+'\n')
    f.write('R21\t:\t'+str(R[21])+'\n')
    f.write('R22\t:\t'+str(R[22])+'\n')
    f.write('R23\t:\t'+str(R[23])+'\n')
    f.write('R24\t:\t'+str(R[24])+'\n')
    f.write('R25\t:\t'+str(R[25])+'\n')
    f.write('R26\t:\t'+str(R[26])+'\n')
    f.write('R27\t:\t'+str(R[27])+'\n')
    f.write('R28\t:\t'+str(R[28])+'\n')
    f.write('R29\t:\t'+str(R[29])+'\n')
    f.write('R30\t:\t'+str(R[30])+'\n')
    f.write('R31\t:\t'+str(R[31])+'\n')
    f.close()
    f=open('data.txt','w')
    f.write('.data'+'\n')
    f1=open('label.txt','r')
    a=int('0x10000000',16)
    for each in f1:
        x=re.split('#',each)
        while "" in x:
            x.remove("")
        while None in x:
            x.remove(None)
        if len(x)<=3:
            break;
        D={str(hex(a)):x[3]}
        f.write(str(D)+'\n')
        a=a+sys.getsizeof(x[3])
    f.write('.text'+'\n')
    f2=open('ref.txt','r')
    b=int('400000',16)
    for each in f2:
        x=re.split('#',each)
        a=int(x[0])
        i=1
        f3=open('Q.s','r')
        for y in f3:
            y=y.strip()
            if i==a:
                z=re.split(',| |\(|\)',y)
                while "" in z:
                    z.remove("")
                while None in z:
                    z.remove(None)
                if len(z)==0:
                    break
                if z[0]=='li':
                    y='addi '+str(z[1])+', 0, '+str(z[2])
                elif z[0]=='mr':
                    y='or '+str(z[1])+', '+str(z[2])+', '+str(z[2])
                elif z[0]=='la':
                    y='addi '+str(z[1])+', '+str(z[2])+', '+str(z[3])
                elif z[0]=='subi':
                    y='addi '+str(z[1])+', '+str(z[2])+', -'+str(z[3])
                elif z[0]=='beq':
                    y='bc '+str(12)+', '+str(10)+', '+str(z[3])
                elif z[0]=='bne':
                    y='bc '+str(4)+', '+str(2)+', '+str(z[3])
                D={str(hex(b)):y}
                f.write(str(D)+'\n')
                b=b+8
            i=i+1
        f3.close()
    f2.close()
    f1.close()
    f.close()
    c='\n'
    i=0
    fp=open('opcode.o','r')
    for each in fp:
        i=i+1
    j=0
    fp.close()
    file=open('label.txt','r')
    for each in file:
        each=re.split('#',each)
        if each[1]=='main':
            k=int(each[0])
            break
    file.close()
    temp='0x400000'
    flag=0
    p=0
    if ch==1:
        file=open('opcodeindex.txt','r')
        for each in file:
            each=re.split('#',each)
            if int(each[0])<=k:
                j=j+1
                continue
            else:
                break
        start=j
        while j<i:
            NIA=int(temp,16)+(j+1)*8
            NIA=hex(NIA)
            fp=open('opcode.o','r')
            k=0
            while k<j:
                each=fp.readline()
                k=k+1
            each=fp.readline()
            op=each[0:6]
            op=int(op,2)
            if op==14:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:30],2)]
                if each[30:32]=='00':
                    R[r[0]]=R[r[1]]+r[2]
                elif each[30:32]=='10':
                    R[r[0]]=R[r[1]]-r[2]
                else:
                    f1=open('label.txt','r')
                    for e in f1:
                        if e[0]==str(r[2]):
                            d=re.split('#',e)
                            if d[2]=='.word':
                                xy=re.split(' |, ',d[3])
                                xy.remove('\n')
                                if len(xy)==1:
                                    R[r[0]]=int(d[3])
                                else:
                                    q=0
                                    while q<p:
                                        xy.remove(xy[0])
                                        q=q+1
                                    R[r[0]]=int(xy[0])
                                    p=p+1
                            elif len(d)>3:
                                R[r[0]]=d[3]
                                break
                    f1.close()
            elif op==31 and int(each[22:31],2)==266:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:21],2)]
                R[r[0]]=R[r[1]]+R[r[2]]
            elif op==31 and int(each[21:31],2)==444:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:21],2)]
                R[r[1]]=R[r[0]] | R[r[2]]
            elif op==17:
                if each[31]=='0':
                    print R[4]
                elif each[31]=='1':
                    R[3]=input()
            elif op==19:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:32],2)]
                if r[0]==4 and r[1]==2:
                    if CR=='1000' or CR=='0100':
                        file=open('opcodeindex.txt','r')
                        k=0
                        for ea in file:
                            ea=re.split('#',ea)
                            if int(ea[0])<r[2]:
                                k=k+1
                                continue
                        j=k
                        file.close()
                        continue
                elif r[0]==12 and r[1]==10:
                    if CR=='0010':
                        file=open('opcodeindex.txt','r')
                        k=0
                        for ea in file:
                            ea=re.split('#',ea)
                            if int(ea[0])<r[2]:
                                k=k+1
                                continue
                        j=k
                        file.close()
                        continue
                elif each[31]=='1':
                    NIA=LR
                    j=start
                    continue
            elif op==18 and each[30:32]=='01':
                file=open('opcodeindex.txt','r')
                k=0
                for ea in file:
                    ea=re.split('#',ea)
                    if int(ea[0])<r[2]:
                        k=k+1
                        continue
                start=j+1
                j=k
                LR=NIA
                file.close()
                continue
            elif op==31 and int(each[21:32],2)==0:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:21],2)]
                if r[0]==7:
                    CR='0000'
                    if R[r[1]]>R[r[2]]:
                        CR='0100'
                    elif R[r[1]]<R[r[2]]:
                        CR='1000'
                    elif R[r[1]]==R[r[2]]:
                        CR='0010'
            elif op==36:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:31],2)]
                R[r[2]+r[1]]=R[r[0]]
            fp.close()
            j=j+1
            CIA=NIA
    elif ch==0:
        print 'Simulating Step By Step'
        print 'Enter 0 to stop simulation and 1 to continue'
        c='1'
        file=open('opcodeindex.txt','r')
        for each in file:
            each=re.split('#',each)
            if int(each[0])<=k:
                j=j+1
                continue
            else:
                break
        start=j
        while j<i:
            NIA=int(temp,16)+(j+1)*8
            NIA=hex(NIA)
            c=str(input('1/0 : '))
            if c=='0':
                break
            fp=open('opcode.o','r')
            k=0
            while k<j:
                each=fp.readline()
                k=k+1
            each=fp.readline()
            op=each[0:6]
            op=int(op,2)
            if op==14:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:30],2)]
                if each[30:32]=='00':
                    R[r[0]]=R[r[1]]+r[2]
                elif each[30:32]=='10':
                    R[r[0]]=R[r[1]]-r[2]
                else:
                    f1=open('label.txt','r')
                    for e in f1:
                        if e[0]==str(r[2]):
                            d=re.split('#',e)
                            if d[2]=='.word':
                                xy=re.split(' |, ',d[3])
                                xy.remove('\n')
                                if len(xy)==1:
                                    R[r[0]]=int(d[3])
                                else:
                                    q=0
                                    while q<p:
                                        xy.remove(xy[0])
                                        q=q+1
                                    R[r[0]]=int(xy[0])
                                    p=p+1
                            elif len(d)>3:
                                R[r[0]]=d[3]
                                break
                    f1.close()
            elif op==31 and int(each[22:31],2)==266:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:21],2)]
                R[r[0]]=R[r[1]]+R[r[2]]
            elif op==31 and int(each[21:31],2)==444:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:21],2)]
                R[r[1]]=R[r[0]] | R[r[2]]
            elif op==17:
                if each[31]=='0':
                    print R[4]
                elif each[31]=='1':
                    R[3]=input()
            elif op==19:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:32],2)]
                if r[0]==4 and r[1]==2:
                    if CR=='1000' or CR=='0100':
                        file=open('opcodeindex.txt','r')
                        k=0
                        for ea in file:
                            ea=re.split('#',ea)
                            if int(ea[0])<r[2]:
                                k=k+1
                                continue
                        j=k
                        file.close()
                        continue
                elif r[0]==12 and r[1]==10:
                    if CR=='0010':
                        file=open('opcodeindex.txt','r')
                        k=0
                        for ea in file:
                            ea=re.split('#',ea)
                            if int(ea[0])<r[2]:
                                k=k+1
                                continue
                        j=k
                        file.close()
                        continue
                elif each[31]=='1':
                    NIA=LR
                    j=start
                    continue
            elif op==18 and each[30:32]=='01':
                file=open('opcodeindex.txt','r')
                k=0
                for ea in file:
                    ea=re.split('#',ea)
                    if int(ea[0])<r[2]:
                        k=k+1
                        continue
                start=j+1
                j=k
                LR=NIA
                file.close()
                continue
            elif op==31 and int(each[21:32],2)==0:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:21],2)]
                if r[0]==7:
                    CR='0000'
                    if R[r[1]]>R[r[2]]:
                        CR='0100'
                    elif R[r[1]]<R[r[2]]:
                        CR='1000'
                    elif R[r[1]]==R[r[2]]:
                        CR='0010'
            elif op==36:
                r=[int(each[6:11],2),int(each[11:16],2),int(each[16:31],2)]
                R[r[2]+r[1]]=R[r[0]]
            f=open('memory.txt','w')
            f.write('CIA\t:\t'+str(CIA)+'\n')
            f.write('NIA\t:\t'+str(NIA)+'\n')
            f.write('SR\t:\t'+str(SR)+'\n')
            f.write('LR\t:\t'+str(LR)+'\n')
            f.write('CR\t:\t'+str(CR)+'\n')
            f.write('R0\t:\t'+str(R[0])+'\n')
            f.write('R1\t:\t'+str(R[1])+'\n')
            f.write('R2\t:\t'+str(R[2])+'\n')
            f.write('R3\t:\t'+str(R[3])+'\n')
            f.write('R4\t:\t'+str(R[4])+'\n')
            f.write('R5\t:\t'+str(R[5])+'\n')
            f.write('R6\t:\t'+str(R[6])+'\n')
            f.write('R7\t:\t'+str(R[7])+'\n')
            f.write('R8\t:\t'+str(R[8])+'\n')
            f.write('R9\t:\t'+str(R[9])+'\n')
            f.write('R10\t:\t'+str(R[10])+'\n')
            f.write('R11\t:\t'+str(R[11])+'\n')
            f.write('R12\t:\t'+str(R[12])+'\n')
            f.write('R13\t:\t'+str(R[13])+'\n')
            f.write('R14\t:\t'+str(R[14])+'\n')
            f.write('R15\t:\t'+str(R[15])+'\n')
            f.write('R16\t:\t'+str(R[16])+'\n')
            f.write('R17\t:\t'+str(R[17])+'\n')
            f.write('R18\t:\t'+str(R[18])+'\n')
            f.write('R19\t:\t'+str(R[19])+'\n')
            f.write('R20\t:\t'+str(R[20])+'\n')
            f.write('R21\t:\t'+str(R[21])+'\n')
            f.write('R22\t:\t'+str(R[22])+'\n')
            f.write('R23\t:\t'+str(R[23])+'\n')
            f.write('R24\t:\t'+str(R[24])+'\n')
            f.write('R25\t:\t'+str(R[25])+'\n')
            f.write('R26\t:\t'+str(R[26])+'\n')
            f.write('R27\t:\t'+str(R[27])+'\n')
            f.write('R28\t:\t'+str(R[28])+'\n')
            f.write('R29\t:\t'+str(R[29])+'\n')
            f.write('R30\t:\t'+str(R[30])+'\n')
            f.write('R31\t:\t'+str(R[31])+'\n')
            f.close()
            fp.close()
            CIA=NIA
            j=j+1
    # Loop ends

    f=open('memory.txt','w')
    f.write('CIA\t:\t'+str(CIA)+'\n')
    f.write('NIA\t:\t'+str(NIA)+'\n')
    f.write('SR\t:\t'+str(SR)+'\n')
    f.write('LR\t:\t'+str(LR)+'\n')
    f.write('CR\t:\t'+str(CR)+'\n')
    f.write('R0\t:\t'+str(R[0])+'\n')
    f.write('R1\t:\t'+str(R[1])+'\n')
    f.write('R2\t:\t'+str(R[2])+'\n')
    f.write('R3\t:\t'+str(R[3])+'\n')
    f.write('R4\t:\t'+str(R[4])+'\n')
    f.write('R5\t:\t'+str(R[5])+'\n')
    f.write('R6\t:\t'+str(R[6])+'\n')
    f.write('R7\t:\t'+str(R[7])+'\n')
    f.write('R8\t:\t'+str(R[8])+'\n')
    f.write('R9\t:\t'+str(R[9])+'\n')
    f.write('R10\t:\t'+str(R[10])+'\n')
    f.write('R11\t:\t'+str(R[11])+'\n')
    f.write('R12\t:\t'+str(R[12])+'\n')
    f.write('R13\t:\t'+str(R[13])+'\n')
    f.write('R14\t:\t'+str(R[14])+'\n')
    f.write('R15\t:\t'+str(R[15])+'\n')
    f.write('R16\t:\t'+str(R[16])+'\n')
    f.write('R17\t:\t'+str(R[17])+'\n')
    f.write('R18\t:\t'+str(R[18])+'\n')
    f.write('R19\t:\t'+str(R[19])+'\n')
    f.write('R20\t:\t'+str(R[20])+'\n')
    f.write('R21\t:\t'+str(R[21])+'\n')
    f.write('R22\t:\t'+str(R[22])+'\n')
    f.write('R23\t:\t'+str(R[23])+'\n')
    f.write('R24\t:\t'+str(R[24])+'\n')
    f.write('R25\t:\t'+str(R[25])+'\n')
    f.write('R26\t:\t'+str(R[26])+'\n')
    f.write('R27\t:\t'+str(R[27])+'\n')
    f.write('R28\t:\t'+str(R[28])+'\n')
    f.write('R29\t:\t'+str(R[29])+'\n')
    f.write('R30\t:\t'+str(R[30])+'\n')
    f.write('R31\t:\t'+str(R[31])+'\n')
    f.close()

# Question - 1
### A program to access two values stored in .data section, compute the sum and store it.
    .data
    X:  .word 5
    Y:  .word 10
    SUM:  .word 0
    .text
    main:
      la 4, 0(X)
      addi 5, 4, 0
      li 4, 0

      la 4, 0(Y)
      addi 6, 4, 0
      li 4, 0

      la 4, 0(SUM)
      add 4, 5, 6
      stw 4, 0(2)

In [5]:
pass1(1)

In [6]:
pass2(1)

Enter Your Choice : 
 0. Step By Step
 1. Full Execution

##### Full Execution

In [7]:
execute(1)

##### Step-by-step Execution

In [8]:
execute(0)

Simulating Step By Step
Enter 0 to stop simulation and 1 to continue
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1


# Question - 2
### A program to get two values from the user, compute the sum, and print it out, with relevant user prompts accompanying them.
    .data
    n1:  .asciiz "Enter 1st Number : "
    n2:  .asciiz "Enter 2nd Number : "
    sum: .asciiz "The Sum : "
    .text
    main:
      la 4, 0(n1)
      sc 4
      li 4, 0

      li 3, 0
      sc 3
      addi 5,3,0
      li 3,0

      la 4, 0(n2)
      sc 4
      li 4, 0

      li 3, 0
      sc 3
      addi 6, 3, 0
      li 3, 0

      add 8, 5, 6

      la 4, 0(sum)
      sc 4
      li 4, 0

      addi 4, 8, 0
      sc 4
      li 4, 0

In [9]:
pass1(2)

In [10]:
pass2(2)

Enter Your Choice : 
 0. Step By Step
 1. Full Execution

##### Full Execution

In [11]:
execute(1)

Enter 1st Number : 

10
Enter 2nd Number : 

20
The Sum : 

30


# Step-by-step Execution

In [12]:
execute(0)

Simulating Step By Step
Enter 0 to stop simulation and 1 to continue
1/0 : 1
1/0 : 1
Enter 1st Number : 

1/0 : 1
1/0 : 1
1/0 : 1
80
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
Enter 2nd Number : 

1/0 : 1
1/0 : 1
1/0 : 1
20
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
The Sum : 

1/0 : 1
1/0 : 1
1/0 : 1
100
1/0 : 1


# Question - 3
### A program to compute the sum of an array of numbers stored in .data X, with length of array stored in .data N. This program uses loop to iterate through the array items.
    .data
    N:  .word 5
    X:  .word 1, 2, 33, 41 , 5
    Sum:  .asciiz "The Sum : "
    .text

    main:
      la 5, 0(N)
      li 7, 0
      li 0, 0
      loop:
        la 6, 0(X)
        add 7, 7, 6
        subi 5, 5, 1
        bne 5, 0, loop
      li 4, 0
      la 4, 0(Sum)
      sc 4
      li 4, 0

      addi 4, 7, 0
      sc 4
      li 4, 0

In [22]:
pass1(3)

In [23]:
pass2(3)

Enter Your Choice : 
 0. Step By Step
 1. Full Execution

##### Full Execution

In [24]:
execute(1)

The Sum : 

82


##### Step-by-step Execution

In [25]:
execute(0)

Simulating Step By Step
Enter 0 to stop simulation and 1 to continue
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
The Sum : 

1/0 : 1
1/0 : 1
1/0 : 1
82
1/0 : 1


# Question - 4
### A program to compute the sum of an array of numbers stored in .data X, with length of array stored in .data N. This program uses recursion to iterate through the array items. This implements both branching and jumping instructions.
    .data
    N:  .word 5
    X:  .word 1, 2, 33, 41 , 5
    Sum:  .asciiz "The Sum : "
    .text

    loop:
      la 6, 0(X)
      add 7, 7, 6
      subi 5, 5, 1
      bne 5, 0, loop
      bclr

    main:
      la 5, 0(N)
      li 7, 0
      li 0, 0

      bl loop

      li 4, 0
      la 4, 0(Sum)
      sc 4
      li 4, 0

      addi 4, 7, 0
      sc 4
      li 4, 0

In [30]:
pass1(4)

In [31]:
pass2(4)

Enter Your Choice : 
 0. Step By Step
 1. Full Execution

##### Full Execution

In [32]:
execute(1)

The Sum : 

216


##### Step-by-step Execution

In [33]:
execute(0)

Simulating Step By Step
Enter 0 to stop simulation and 1 to continue
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
1/0 : 1
The Sum : 

1/0 : 1
1/0 : 1
1/0 : 1
216
1/0 : 1
